# Cook-It-Up-With-the-Fantastic-Four

Python depends on packages for most of its functionality; these can be either built-in (such as sys), or third-party (like all the packages below). Either way you need to import the packages you need before using them.

In [2]:
%matplotlib inline
# See all the "as ..." contructs? They're just aliasing the package names.
# That way we can call methods like plt.plot() instead of matplotlib.pyplot.plot().
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
import time
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

In [3]:
dftouse=pd.read_csv("submission.csv")
dftouse.head()

,Unnamed: 0,id,ingredients_clean_string,cuisine
0,4987,5,"mushrooms , chopped onion , tomato sauce , che...",mexican
1,9232,7,"minced garlic , brown rice , sour cream , chic...",indian
2,9638,11,"lime juice , sesame oil , garlic cloves , fish...",vietnamese
3,4927,12,"sugar , vanilla extract , corn starch , coffee...",italian
4,3280,13,"frozen pie crust , bourbon whiskey , powdered ...",southern_us


### Create dataframes for Train Data

In [13]:
import json
import pandas as pd

traindf = pd.read_json('train.json')
traindf.head()



,cuisine,id,ingredients
0,greek,10259,"[romaine lettuce, black olives, grape tomatoes..."
1,southern_us,25693,"[plain flour, ground pepper, salt, tomatoes, g..."
2,filipino,20130,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,indian,22213,"[water, vegetable oil, wheat, salt]"
4,indian,13162,"[black pepper, shallots, cornflour, cayenne pe..."
